In [1]:
%load_ext autoreload
%autoreload 2
from functools import partial

import keras

from datasets import example_datasets, to_numpy
from models import mixture_poissons,location_specific_linear, CustomPenalizedMixtureDecisionModel
from metrics import mixture_poi_loss, get_bpr_loss_func, mix_bpr, get_penalized_bpr_loss_func_mix, cross_ratio_decision, get_perturbed_bpr_func
from experiments import training_loop, training_loop_score_function_trick, score_function_trick, overall_gradient_calculation
from plotting_funcs import plot_losses, plot_frontier

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

import matplotlib.pyplot as plt
import resource
import datetime

2024-05-12 14:16:35.456275: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 14:16:35.499980: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 14:16:35.500009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 14:16:35.501076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 14:16:35.508539: I tensorflow/core/platform/cpu_feature_guar

In [2]:
seed=360
num_components=4
learning_rate = 0.05
epochs=200
outdir = '/cluster/home/kheuto01/testdir'
penalty = 5000
threshold = 0.55
K=4
do_only=True
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500
perturbed_sigma=0.1
num_score_func_samples=5
batch_size=50

In [3]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed, batch_size=batch_size)
train_X_THS, train_y_TS = to_numpy(train_dataset)
val_X_THS, val_y_TS = to_numpy(val_dataset)
input_shape = (H,S)

bpr_K = get_perturbed_bpr_func(K, sigma=perturbed_sigma)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


2024-05-12 14:16:43.938036: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-12 14:16:43.938070: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2024-05-12 14:16:43.938078: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: s1cmp008.pax.tufts.edu
2024-05-12 14:16:43.938143: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-05-12 14:16:43.938167: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-05-12 14:16:43.938172: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [4]:
model, mix_weights  = mixture_poissons(location_specific_linear, input_shape, num_components=num_components)

In [5]:
@tf.function
def build_distribution(prob_params_BSK, mixture_weights_KS):
    # my fault that model returns KS instead of SK
    mixture_weights_SK = tf.transpose(mixture_weights_KS, perm=[1,0])

    #  could create a custom model class that returns 
    # the appropriate tfp.dist given outputs
    mix = tfp.distributions.MixtureSameFamily(
        mixture_distribution=tfp.distributions.Categorical(probs=mixture_weights_SK),#+1e-13),
        components_distribution = tfp.distributions.Poisson(rate=prob_params_BSK+1e-13))

    return mix

@tf.function
def get_samples(mix, num_score_func_samples):
    sample_y_MBS = mix.sample(num_score_func_samples)
    return sample_y_MBS

@tf.function
def get_sample_log_probs(mix, sample_y_MBS):
    sample_log_probs_MBS = mix.log_prob(sample_y_MBS)
    return sample_log_probs_MBS

@tf.function
def get_expected_decision(sample_decisions_MBS):
    expected_decisions_BS = tf.reduce_mean(sample_decisions_MBS, axis=0)
    return expected_decisions_BS

@tf.function
def get_bpr_B(y_BS, expected_decisions_BS):
    bpr_B = bpr_K(y_BS, expected_decisions_BS)
    return bpr_B

@tf.function
def get_observed_log_prob(mix, y_BS):
    observed_log_prob_BS = mix.log_prob(y_BS)
    return observed_log_prob_BS

@tf.function
def get_loss(observed_log_prob_BS, threshold, bpr_B):

    loss_B = -tf.reduce_sum(observed_log_prob_BS, axis=-1)

    violate_threshold_flag_B = tf.cast(tf.greater(threshold,
                                                bpr_B),
                                        tf.float32)
    loss_B += penalty * violate_threshold_flag_B *(threshold - bpr_B)
    return loss_B

In [ ]:
@tf.function
def do_it_all(prob_params_BSK, mixture_weights_KS):
    # my fault that model returns KS instead of SK
    mixture_weights_SK = tf.transpose(mixture_weights_KS, perm=[1,0])

    #  could create a custom model class that returns 
    # the appropriate tfp.dist given outputs
    mix = tfp.distributions.MixtureSameFamily(
        mixture_distribution=tfp.distributions.Categorical(probs=mixture_weights_SK),#+1e-13),
        components_distribution = tfp.distributions.Poisson(rate=prob_params_BSK+1e-13))

    # add constant to avoid log 0
    sample_y_MBS = mix.sample(num_score_func_samples)#+1e-13

    sample_log_probs_MBS = mix.log_prob(sample_y_MBS)
    sample_decisions_MBS = cross_ratio_decision(sample_y_MBS)
    expected_decisions_BS = tf.reduce_mean(sample_decisions_MBS, axis=0)
    bpr_B = bpr_K(y_BS, expected_decisions_BS)
    observed_log_prob_BS = mix.log_prob(y_BS)


    loss_B = -tf.reduce_sum(observed_log_prob_BS, axis=-1)

    violate_threshold_flag_B = tf.cast(tf.greater(threshold,
                                                bpr_B),
                                        tf.float32)
    loss_B += penalty * violate_threshold_flag_B *(threshold - bpr_B)

    return loss_B, sample_log_probs_MBS, sample_decisions_MBS, expected_decisions_BS

In [6]:
#check numerics
tf.debugging.disable_check_numerics()

losses = {}
losses['train'] = {}
losses['val'] ={}
losses['train']['loss']=[]
losses['train']['nll']=[]
losses['train']['bpr']=[]
losses['val']['loss']=[]
losses['val']['nll']=[]
losses['val']['bpr']=[]
verbose=True
for epoch in range(10):

        if verbose:
            print(f'Epoch {epoch}')
        else:
            if epoch % 10 == 0:
                print(f'Epoch {epoch}')

        batch_losses = {'train': {'loss': [], 'nll': [], 'bpr': []}}

        for step, (x_BHS, y_BS) in enumerate(train_dataset):
            print(step)
            
            with tf.GradientTape() as jacobian_tape, tf.GradientTape() as loss_tape:
                
                prob_params_BSK, mixture_weights_KS = model(x_BHS, training=True)
                print('model evaled')

                mix = build_distribution(prob_params_BSK, mixture_weights_KS)
                print('got mix')
                sample_y_MBS = get_samples(mix, num_score_func_samples)
                print('got samples')
                sample_log_probs_MBS = get_sample_log_probs(mix, sample_y_MBS)
                print('got probs of samples')
                sample_decisions_MBS = cross_ratio_decision(sample_y_MBS)
                print('got decisions')
                expected_decisions_BS = get_expected_decision(sample_decisions_MBS)
                print('got expected decisions')


                bpr_B = get_bpr_B(y_BS, expected_decisions_BS)
                print('print got bpr')
                observed_log_prob_BS = get_observed_log_prob(mix, y_BS)
                loss_B = get_loss(observed_log_prob_BS, threshold, bpr_B)

                '''
                # my fault that model returns KS instead of SK
                mixture_weights_SK = tf.transpose(mixture_weights_KS, perm=[1,0])

                #  could create a custom model class that returns 
                # the appropriate tfp.dist given outputs
                mix = tfp.distributions.MixtureSameFamily(
                    mixture_distribution=tfp.distributions.Categorical(probs=mixture_weights_SK),#+1e-13),
                    components_distribution = tfp.distributions.Poisson(rate=prob_params_BSK+1e-13))

                # add constant to avoid log 0
                sample_y_MBS = mix.sample(num_score_func_samples)#+1e-13

                sample_log_probs_MBS = mix.log_prob(sample_y_MBS)
                sample_decisions_MBS = cross_ratio_decision(sample_y_MBS)
                expected_decisions_BS = tf.reduce_mean(sample_decisions_MBS, axis=0)
                bpr_B = bpr_K(y_BS, expected_decisions_BS)
                observed_log_prob_BS = mix.log_prob(y_BS)

                loss_B = tf.zeros_like(bpr_B)

                if True:
                    loss_B -= tf.reduce_sum(observed_log_prob_BS, axis=-1)
                if True:
                    violate_threshold_flag_B = tf.cast(tf.greater(threshold,
                                                                bpr_B),
                                                        tf.float32)
                    loss_B += penalty * violate_threshold_flag_B *(threshold - bpr_B)
                '''

            # The lowercase "p" signifies that these are lists of length P, P = number of trainable variables
            print('before jac')
            jacobian_pMBS = jacobian_tape.jacobian(sample_log_probs_MBS, model.trainable_weights)
            print('after jac')
            param_gradient_pBS = [score_function_trick(j, sample_decisions_MBS) for j in jacobian_pMBS]
            print('before grad')
            loss_gradients_BS = loss_tape.gradient(loss_B, expected_decisions_BS)
            print('after grad')
            overall_gradient = [overall_gradient_calculation(g, loss_gradients_BS) for g in param_gradient_pBS]
            print('grads combined')

            # Run one step of gradient descent by updating
            # the value of the variables to minimize the loss.
            optimizer.apply_gradients(zip(overall_gradient, model.trainable_weights))
            


            batch_losses['train']['loss'].append(tf.reduce_mean(loss_B))
            batch_losses['train']['nll'].append(tf.reduce_mean(tf.reduce_sum(observed_log_prob_BS, axis=-1)))
            batch_losses['train']['bpr'].append(tf.reduce_mean(bpr_B))
            print(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
            
            
        losses['train']['loss'].append(tf.reduce_mean(batch_losses['train']['loss']))
        losses['train']['nll'].append(tf.reduce_mean(batch_losses['train']['nll']))
        losses['train']['bpr'].append(tf.reduce_mean(batch_losses['train']['bpr']))

        if verbose:
            # print all metrics
            print(f'Loss: {losses["train"]["loss"][-1]}')
            print(f'NLL: {losses["train"]["nll"][-1]}')
            print(f'BPR: {losses["train"]["bpr"][-1]}')

        '''
        
        '''
                
            

            


Epoch 0
0
model evaled
got mix
got samples
got probs of samples
got decisions
got expected decisions
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got partial
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got partial
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
print got bpr
before jac
after jac
before grad
after grad
grads combined
1140668
1
model evaled
got mix
got samples
got probs of samples
got decisions
got expected decisions
print got bpr
befor

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp

# Define a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1),
    tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1)),
])

# Define some data
x = tf.random.normal(shape=(100, 1))
y = tf.random.normal(shape=(100, 1))

# Define a loss function
def nll(y_true, y_pred):
    return -y_pred.log_prob(y_true)

# Compile the model
model.compile(optimizer='adam', loss=nll)

# Fit the model
model.fit(x, y, epochs=10)

# Now, let's differentiate through samples and log probabilities
with tf.GradientTape() as tape:
    # Forward pass
    y_pred = model(x)
    # Compute loss
    loss = nll(y, y_pred)

# Get the gradients of the loss with respect to the model's trainable variables
gradients = tape.gradient(loss, model.trainable_weights)

# Print the gradients
for grad in gradients:
    print(grad)

2024-05-12 14:26:46.462884: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 14:26:46.507144: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 14:26:46.507170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 14:26:46.508242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 14:26:46.515890: I tensorflow/core/platform/cpu_feature_guar

Epoch 1/10


2024-05-12 14:26:54.816543: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-12 14:26:54.816577: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2024-05-12 14:26:54.816584: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: s1cmp008.pax.tufts.edu
2024-05-12 14:26:54.816647: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-05-12 14:26:54.816671: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-05-12 14:26:54.816676: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


4/4 [==============================] - 0s 3ms/step - loss: 2.7334
Epoch 2/10
4/4 [==============================] - 0s 1ms/step - loss: 2.7260
Epoch 3/10
4/4 [==============================] - 0s 1ms/step - loss: 2.7180
Epoch 4/10
4/4 [==============================] - 0s 1ms/step - loss: 2.7110
Epoch 5/10
4/4 [==============================] - 0s 2ms/step - loss: 2.7039
Epoch 6/10
4/4 [==============================] - 0s 1ms/step - loss: 2.6962
Epoch 7/10
4/4 [==============================] - 0s 1ms/step - loss: 2.6887
Epoch 8/10
4/4 [==============================] - 0s 1ms/step - loss: 2.6814
Epoch 9/10
4/4 [==============================] - 0s 1ms/step - loss: 2.6733
Epoch 10/10
4/4 [==============================] - 0s 1ms/step - loss: 2.6666
tf.Tensor([[183.6322]], shape=(1, 1), dtype=float32)
tf.Tensor([8.803156], shape=(1,), dtype=float32)


In [6]:
y_pred + 1

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[-1.484607  ],
       [ 0.08382666],
       [ 3.205427  ],
       [ 1.8569267 ],
       [ 1.0521313 ],
       [ 3.26027   ],
       [ 0.8267242 ],
       [-0.04156196],
       [ 3.305962  ],
       [ 0.19652712],
       [ 2.3469286 ],
       [-0.21311045],
       [ 1.7456309 ],
       [ 0.1669746 ],
       [-0.92454946],
       [ 2.629715  ],
       [ 2.7455034 ],
       [ 0.55555665],
       [ 2.3121119 ],
       [ 2.9785523 ],
       [-0.94783545],
       [-0.26307166],
       [ 3.2826483 ],
       [ 2.024968  ],
       [ 1.230127  ],
       [-1.3259962 ],
       [-0.6261064 ],
       [-3.4701219 ],
       [ 4.0360813 ],
       [ 3.9916093 ],
       [ 2.1007195 ],
       [ 3.0485828 ],
       [ 3.3723094 ],
       [ 0.7909816 ],
       [ 2.1264493 ],
       [-2.4550061 ],
       [ 3.5258813 ],
       [ 3.9761531 ],
       [ 1.7272794 ],
       [ 0.49896324],
       [-0.35074925],
       [ 1.4300237 ],
       [ 3.670957  ],
    